<p style = 'color:#E6E4E7 ; background-color:#00561B ; text-align:center ; font-size: 300%'> Kayak project - Get hotel data for Eguisheim

<p style = 'color:#00561B ; background-color:#E6E4E7 ; text-align:center ; font-size: 150%'> Project prepared by Stephanie Cotineau - #dsmpt-Paris-08

In [1]:
!pip install Scrapy -q

In [2]:
import os # For manipulate file 
import logging # For display logs
import scrapy # Import scrapy & scrapy.crawler
from scrapy.crawler import CrawlerProcess
import re

In [3]:
# Name of the file where the results will be saved
filename = "Eguisheim_multipage.json"

# If file already exists, delete it before crawling (because Scrapy will 
# concatenate the last and new results otherwise)
if filename in os.listdir('results_hotels/'):
        os.remove('results_hotels/' + filename)

# Declare a new CrawlerProcess with some settings
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'
process = CrawlerProcess(settings = {
    'USER_AGENT': user_agent,
    'LOG_LEVEL': logging.ERROR,
    "FEEDS": {
        'results_hotels/' + filename : {"format": "json"},
    }
})

In [4]:
from bs4 import BeautifulSoup 
import requests

In [5]:
# Eguisheim - 5 first pages
offset = 0
link_offset = []
while offset != 125:
    url = f"https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=c82459aab844011f&search_selected=true&ss_raw=eguisheim&offset={offset}"
    #print(url)
    offset = offset + 25
    link_offset.append(url)

In [6]:
link_offset

['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=c82459aab844011f&search_selected=true&ss_raw=eguisheim&offset=0',
 'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1g

In [7]:
class EguisheimSpider(scrapy.Spider) :
    # Name of spider
    name = 'Eguisheim'
    
    # Starting URL
    start_urls = ['https://www.booking.com/']
    
    def start_requests(self):
        for d in ['https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=c82459aab844011f&search_selected=true&ss_raw=eguisheim&offset=0',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=c82459aab844011f&search_selected=true&ss_raw=eguisheim&offset=25',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=c82459aab844011f&search_selected=true&ss_raw=eguisheim&offset=50',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=c82459aab844011f&search_selected=true&ss_raw=eguisheim&offset=75',
                  'https://www.booking.com/searchresults.fr.html?label=gen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB&sid=7a5ce1be69aa1fde7be9ba8dc5ddd002&sb=1&sb_lp=1&src=index&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Findex.fr.html%3Flabel%3Dgen173nr-1DCAEoggI46AdIM1gEaE2IAQGYAQ24AQfIAQzYAQPoAQGIAgGoAgO4AtSPw5QGwAIB0gIkMGUzNDM4MTQtYTBiZi00ZjIxLThkNTItYzY4YzYyOGJiZDY22AIE4AIB%26sid%3D7a5ce1be69aa1fde7be9ba8dc5ddd002%26sb_price_type%3Dtotal%26%26&ss=Eguisheim%2C+Alsace%2C+France&is_ski_area=0&checkin_year=&checkin_month=&checkout_year=&checkout_month=&group_adults=2&group_children=0&no_rooms=1&b_h4u_keep_filters=&from_sf=1&ac_position=0&ac_langcode=fr&ac_click_type=b&dest_id=-1425030&dest_type=city&place_id_lat=48.04231&place_id_lon=7.30543&search_pageview_id=c82459aab844011f&search_selected=true&ss_raw=eguisheim&offset=100']:
            yield scrapy.Request(d,callback=self.search)
    
    # Callback used after search launched
    def search(self, response):
        hotels = response.css('div.a826ba81c4.fe821aea6c.fa2f36ad22.afd256fc79.d08f526e0d.ed11e24d01.da89aeb942')
        for hotel in hotels:
            yield {
                'location' : hotel.css('span.f4bd0794db.b4273d69aa::text').get(),
                'hotel_url' : hotel.css('a.e13098a59f').attrib['href'],
                'hotel_name' : hotel.css('div.fcab3ed991.a23c043802::text').get(),
                'hotel_rating' : hotel.css('div.b5cd09854e.d10a6220b4::text').get(),
                'hotel_description_short' : hotel.css('div.d8eab2cf7f::text').get()
            }

In [8]:
# Start the crawling using the spider defined above
process.crawl(EguisheimSpider)
process.start()

In [9]:
import pandas as pd

In [10]:
eguisheim = pd.read_json('results_hotels/Eguisheim_multipage.json')

In [11]:
len(eguisheim)

47

In [12]:
eguisheim.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short
0,Eguisheim,https://www.booking.com/hotel/fr/gite-l-39-aub...,Gîte l'Aubaine,"8,9",Hébergement géré par un particulier
1,Eguisheim,https://www.booking.com/hotel/fr/la-legende-de...,La Légende des 2 Pierres,"9,2","Situé dans le pittoresque village d'Eguisheim,..."
2,Eguisheim,https://www.booking.com/hotel/fr/gite-le-petit...,Gite Le Petit Malsbach Eguisheim,"9,4",Hébergement géré par un particulier
3,Eguisheim,https://www.booking.com/hotel/fr/elsass-design...,Elsass Design Hygge,"9,4",Hébergement géré par un particulier
4,Eguisheim,https://www.booking.com/hotel/fr/le-gite-de-ni...,Le gîte de Ninon et Léon.,"8,7",Hébergement géré par un particulier
5,Eguisheim,https://www.booking.com/hotel/fr/le-petit-nid-...,Le Petit Nid,"9,3",Hébergement géré par un particulier
6,Eguisheim,https://www.booking.com/hotel/fr/appartement-e...,Gite Edelweiss à Eguisheim,"9,8",Le Gite Edelweiss à Eguisheim propose des hébe...
7,Eguisheim,https://www.booking.com/hotel/fr/chez-jade-et-...,Chez Jade et Ambre,"8,5",Hébergement géré par un particulier
8,Eguisheim,https://www.booking.com/hotel/fr/gite-st-leon....,Gite St Léon à Eguisheim,"8,4","Doté d'un restaurant, le Gite St Léon à Eguish..."
9,Eguisheim,https://www.booking.com/hotel/fr/le-gite-de-ga...,Le gîte de Gab,"9,2",Hébergement géré par un particulier


In [13]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_ll =[]
for link in eguisheim['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    lat_lon = soup_link.find_all(id = "hotel_address")
    for ll in lat_lon : 
        #print(ll['data-atlas-latlng'])
        result_ll.append(ll['data-atlas-latlng'])

In [14]:
len(result_ll)

47

In [15]:
eguisheim['Latitude_Longitude'] = result_ll

In [16]:
lat = []
lon = []

# For each row in a varible,
for row in eguisheim['Latitude_Longitude']:
    # Try to,
    try:
        # Split the row by comma and append
        # everything before the comma to lat
        lat.append(row.split(',')[0])
        # Split the row by comma and append
        # everything after the comma to lon
        lon.append(row.split(',')[1])
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
eguisheim['latitude'] = lat
eguisheim['longitude'] = lon

In [17]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.0) AppleWebKit/535.1 (KHTML, like Gecko) Chrome/13.0.782.112 Safari/535.1'}
result_long_description =[]
for link in eguisheim['hotel_url']:
    page = requests.get(link, headers = headers)
    soup_link = BeautifulSoup(page.content, "lxml")
    long_description = soup_link.find_all(id = "property_description_content") 
    for ld in long_description : 
        result_long_description.append(ld.text)

In [18]:
len(result_long_description)

47

In [19]:
eguisheim['hotel_description_long'] = result_long_description

In [20]:
eguisheim.head(55)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Eguisheim,https://www.booking.com/hotel/fr/gite-l-39-aub...,Gîte l'Aubaine,"8,9",Hébergement géré par un particulier,"48.04370220,7.30599767",48.04370220,7.30599767,"\nSitué à Eguisheim, à seulement 400 mètres du..."
1,Eguisheim,https://www.booking.com/hotel/fr/la-legende-de...,La Légende des 2 Pierres,"9,2","Situé dans le pittoresque village d'Eguisheim,...","48.04299853,7.30448939",48.04299853,7.30448939,\nVous pouvez bénéficier d'une réduction Geniu...
2,Eguisheim,https://www.booking.com/hotel/fr/gite-le-petit...,Gite Le Petit Malsbach Eguisheim,"9,4",Hébergement géré par un particulier,"48.04224054,7.31270510",48.04224054,7.31270510,\nLe Gite Le Petit Malsbach Eguisheim est situ...
3,Eguisheim,https://www.booking.com/hotel/fr/elsass-design...,Elsass Design Hygge,"9,4",Hébergement géré par un particulier,"48.04203000,7.30459800",48.04203000,7.30459800,"\nSitué à Eguisheim, l'Elsass Design Hygge pro..."
4,Eguisheim,https://www.booking.com/hotel/fr/le-gite-de-ni...,Le gîte de Ninon et Léon.,"8,7",Hébergement géré par un particulier,"48.04987400,7.30730100",48.04987400,7.30730100,\nVous pouvez bénéficier d'une réduction Geniu...
5,Eguisheim,https://www.booking.com/hotel/fr/le-petit-nid-...,Le Petit Nid,"9,3",Hébergement géré par un particulier,"48.04338036,7.30217079",48.04338036,7.30217079,"\nSitué à Eguisheim, à seulement 700 mètres du..."
6,Eguisheim,https://www.booking.com/hotel/fr/appartement-e...,Gite Edelweiss à Eguisheim,"9,8",Le Gite Edelweiss à Eguisheim propose des hébe...,"48.04323544,7.30522192",48.04323544,7.30522192,\nLe Gite Edelweiss à Eguisheim propose des hé...
7,Eguisheim,https://www.booking.com/hotel/fr/chez-jade-et-...,Chez Jade et Ambre,"8,5",Hébergement géré par un particulier,"48.04376970,7.30540000",48.04376970,7.30540000,"\nOffrant une vue sur la ville, l'établissemen..."
8,Eguisheim,https://www.booking.com/hotel/fr/gite-st-leon....,Gite St Léon à Eguisheim,"8,4","Doté d'un restaurant, le Gite St Léon à Eguish...","48.04286450,7.30612380",48.04286450,7.30612380,"\nDoté d'un restaurant, le Gite St Léon à Egui..."
9,Eguisheim,https://www.booking.com/hotel/fr/le-gite-de-ga...,Le gîte de Gab,"9,2",Hébergement géré par un particulier,"48.04185420,7.31020455",48.04185420,7.31020455,\nVous pouvez bénéficier d'une réduction Geniu...


In [21]:
eguisheim['hotel_description_long'] = eguisheim['hotel_description_long'].replace('\n','', regex=True)

In [22]:
eguisheim.head(5)

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Eguisheim,https://www.booking.com/hotel/fr/gite-l-39-aub...,Gîte l'Aubaine,"8,9",Hébergement géré par un particulier,"48.04370220,7.30599767",48.04370220,7.30599767,"Situé à Eguisheim, à seulement 400 mètres du m..."
1,Eguisheim,https://www.booking.com/hotel/fr/la-legende-de...,La Légende des 2 Pierres,"9,2","Situé dans le pittoresque village d'Eguisheim,...","48.04299853,7.30448939",48.04299853,7.30448939,Vous pouvez bénéficier d'une réduction Genius ...
2,Eguisheim,https://www.booking.com/hotel/fr/gite-le-petit...,Gite Le Petit Malsbach Eguisheim,"9,4",Hébergement géré par un particulier,"48.04224054,7.31270510",48.04224054,7.31270510,Le Gite Le Petit Malsbach Eguisheim est situé ...
3,Eguisheim,https://www.booking.com/hotel/fr/elsass-design...,Elsass Design Hygge,"9,4",Hébergement géré par un particulier,"48.04203000,7.30459800",48.04203000,7.30459800,"Situé à Eguisheim, l'Elsass Design Hygge propo..."
4,Eguisheim,https://www.booking.com/hotel/fr/le-gite-de-ni...,Le gîte de Ninon et Léon.,"8,7",Hébergement géré par un particulier,"48.04987400,7.30730100",48.04987400,7.30730100,Vous pouvez bénéficier d'une réduction Genius ...


In [23]:
eguisheim['hotel_description_long'][15]

"Gite GéraniumLe *** propose un hébergement à Eguisheim, à 100 mètres du marché de Noël.Cet appartement comprend 2 chambres, une télévision à écran plat et une cuisine.L'aéroport de Bâle-Mulhouse-Fribourg, le plus proche, est implanté à 52 km. "

In [24]:
# Clean location column
eguisheim['location'] = ["Eguisheim" if value!="Eguisheim" else value for value in eguisheim["location"]]

In [25]:
eguisheim.head()

,location,hotel_url,hotel_name,hotel_rating,hotel_description_short,Latitude_Longitude,latitude,longitude,hotel_description_long
0,Eguisheim,https://www.booking.com/hotel/fr/gite-l-39-aub...,Gîte l'Aubaine,"8,9",Hébergement géré par un particulier,"48.04370220,7.30599767",48.04370220,7.30599767,"Situé à Eguisheim, à seulement 400 mètres du m..."
1,Eguisheim,https://www.booking.com/hotel/fr/la-legende-de...,La Légende des 2 Pierres,"9,2","Situé dans le pittoresque village d'Eguisheim,...","48.04299853,7.30448939",48.04299853,7.30448939,Vous pouvez bénéficier d'une réduction Genius ...
2,Eguisheim,https://www.booking.com/hotel/fr/gite-le-petit...,Gite Le Petit Malsbach Eguisheim,"9,4",Hébergement géré par un particulier,"48.04224054,7.31270510",48.04224054,7.31270510,Le Gite Le Petit Malsbach Eguisheim est situé ...
3,Eguisheim,https://www.booking.com/hotel/fr/elsass-design...,Elsass Design Hygge,"9,4",Hébergement géré par un particulier,"48.04203000,7.30459800",48.04203000,7.30459800,"Situé à Eguisheim, l'Elsass Design Hygge propo..."
4,Eguisheim,https://www.booking.com/hotel/fr/le-gite-de-ni...,Le gîte de Ninon et Léon.,"8,7",Hébergement géré par un particulier,"48.04987400,7.30730100",48.04987400,7.30730100,Vous pouvez bénéficier d'une réduction Genius ...


In [26]:
eguisheim.to_csv("Eguisheim.csv")